### Load libraries

In [1]:
import os
import sys
import numpy as np
import pandas as pd
import datetime
import time
import pickle

from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping
from keras.models import load_model

path_main_folder = '/home/antorosi/Documents/Prediction'
sys.path.append(path_main_folder)

from conso.load_shape_data import load_data_conso, get_uniformed_data_conso, change_granularity, get_x_y_prediction_conso, get_train_test_sets, normalized_dataset, select_variables
from models.feedforward_NN import FeedForward
from aed.detection import get_residuals, get_atypical_score, get_atypical_ds, prediction_conso_plot
from aed.atypical_event import AtypicalEvent, AtypicalEventsList, get_atypical_events_list, apply_ael_to_df, get_confusion_matrix
from aed.verification import *
from aed.multi_period import get_prediction_results, get_aed_results, get_complete_df

/local/home/antorosi/PycharmProjects/KERAS-TS-VENV/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


### Get original data 

In [18]:
# Load
path_data = os.path.join(path_main_folder, 'data')
dict_data_conso = load_data_conso(path_data)

# Uniformization
data_conso_df, dict_colnames_conso = get_uniformed_data_conso(dict_data_conso)

# Granularity from 15 min to 1H
data_conso_df = change_granularity(data_conso_df, granularity="1H")

### Multi period model characteristics

In [4]:
path_models_folder = os.path.join(path_main_folder, 'out', 'cv_model_0')
name_dict_dataset = 'dict_datasets.pickle'

In [5]:
with open(os.path.join(path_models_folder, name_dict_dataset), 'rb') as f:
    dict_datasets = pickle.load(f)

In [6]:
list_models_folder = [el for el in os.listdir(path_models_folder) if el.startswith(tuple(dict_datasets.keys()))]

assert len(list_models_folder) == len(dict_datasets.keys())

### Get predictions for each period 

In [7]:
prediction_results = {}

for name_period in dict_datasets.keys():
    name_model_folder = [el for el in list_models_folder if el.startswith(name_period)][0]

    model = load_model(os.path.join(path_models_folder,name_model_folder,'models','model-best.hdf5'))

    y = dict_datasets[name_period]['dataset']['test']['y']
    y_hat = model.predict(dict_datasets[name_period]['dataset']['test']['x'])
    
    prediction_df = pd.DataFrame({'ds': dict_datasets[name_period]['dict_ds']['test'], 'prediction':y_hat.flatten()})
    residuals_df = get_residuals(y_obs=y, y_hat=y_hat, ds=dict_datasets[name_period]['dict_ds']['test'])
    detector_df = get_atypical_score(residuals_df, mode=2)

    prediction_results[name_period] = {'detector_df': detector_df, 'prediction_df': prediction_df}

### Get atypical events for each periods and a given threshold

In [8]:
threshold = 0.98
aed_results = {}

In [9]:
for name_period, el in prediction_results.items():
    detector_df = el['detector_df']
    
    atypical_df = get_atypical_ds(detector_df, threshold)
    events_list_model = get_atypical_events_list(atypical_df, atypical_name = name_period)
    
    aed_results[name_period] = {'atypical_df': atypical_df, 'ael_model': events_list_model}

In [10]:
# get full ae list
ael_full_model = AtypicalEventsList()

for name_period, el in aed_results.items():
    ael_model = el['ael_model']
    ael_full_model = ael_full_model.get_union(ael_model)

### Comparing results with reference

In [11]:
# Loading list
path_data = os.path.join("/local/home/antorosi/Documents/Prediction/data")
with open(os.path.join(path_data, 'ae_reference_list_2013-2016' + '.pickle'), 'rb') as f:
    ael_reference= pickle.load(f)

In [12]:
ael_results = ael_reference.strict_comparison(ael_full_model)

In [13]:
nb_events = 1461

In [14]:
dict_confusion_matrix = get_confusion_matrix(ael_results, nb_events)

In [15]:
dict_confusion_matrix

{'a': 36, 'b': 9, 'c': 8, 'd': 1408}

In [16]:
get_all_scores(dict_confusion_matrix)

{'csi': 0.6792452830188679,
 'f': 0.006351446718419196,
 'far': 0.2,
 'gss': 0.6708281976859767,
 'h': 0.8181818181818182,
 'hss': 0.8029888394450664,
 'pc': 0.9883641341546886,
 'pss': 0.811830371463399}

### Plotting results 

In [17]:
atypical_full_df = pd.DataFrame()
prediction_full_df = pd.DataFrame()

for name_period in prediction_results.keys():
    atypical_full_df = atypical_full_df.append(aed_results[name_period]['atypical_df'], ignore_index=True)
    prediction_full_df = prediction_full_df.append(prediction_results[name_period]['prediction_df'], ignore_index=True)
    
prediction_full_df = prediction_full_df.sort_values(by ='ds', axis=0)
atypical_full_df = atypical_full_df.sort_values(by ='ds', axis=0)

prediction_full_df = prediction_full_df.reset_index(drop=True)
atypical_full_df = atypical_full_df.reset_index(drop=True)

In [18]:
atypical_full_df = apply_ael_to_df(atypical_full_df, ael_results)

In [19]:
path_plot = '/home/antorosi/Documents/Prediction/out/figures'
name_plot = 'full_period_mode2'

prediction_conso_plot(data_conso_df, atypical_full_df, prediction_full_df, dict_colnames_conso, path_plot, name_plot)

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x1,y2 ]
[ (3,1) x1,y3 ]
[ (4,1) x1,y4 ]



/local/home/antorosi/PycharmProjects/KERAS-TS-VENV/lib/python3.5/site-packages/plotly/offline/offline.py:459: UserWarning:

Your filename `/home/antorosi/Documents/Prediction/out/figures/full_period_mode2` didn't end with .html. Adding .html to the end of your file.



'file:///home/antorosi/Documents/Prediction/out/figures/full_period_mode2.html'

In [22]:
atypical_full_df

,ds,residuals,a_score,is_atypical
0,2013-01-01 00:00:00,283.949219,1.000000,1.0
1,2013-01-01 01:00:00,1347.460938,1.000000,1.0
2,2013-01-01 02:00:00,1432.312500,1.000000,1.0
3,2013-01-01 03:00:00,1075.046875,1.000000,1.0
4,2013-01-01 04:00:00,-487.402344,1.000000,1.0
5,2013-01-01 05:00:00,-1656.550781,1.000000,1.0
6,2013-01-01 06:00:00,-4516.027344,1.000000,1.0
7,2013-01-01 07:00:00,-8402.371094,1.000000,1.0
8,2013-01-01 08:00:00,-12562.980469,1.000000,1.0
9,2013-01-01 09:00:00,-14892.628906,1.000000,1.0


### Multi period prediction and aed results

In [3]:
# model paths
path_models_folder = os.path.join(path_main_folder, 'out', 'cv_model_0')
name_dict_dataset = 'dict_datasets.pickle'

In [5]:
# get prediction results
prediction_results = get_prediction_results(path_models_folder, name_dict_dataset, mode=1)

In [7]:
# get atypical events info
threshold = 0.98
aed_results, ael_full_model = get_aed_results(prediction_results, threshold)

### Comparing results with reference and getting scores 

In [9]:
# Loading list
path_data = os.path.join("/local/home/antorosi/Documents/Prediction/data")
with open(os.path.join(path_data, 'ae_reference_list_2013-2016' + '.pickle'), 'rb') as f:
    ael_reference= pickle.load(f)

In [10]:
ael_results = ael_reference.strict_comparison(ael_full_model)

In [11]:
nb_events = 1461

In [12]:
dict_confusion_matrix = get_confusion_matrix(ael_results, nb_events)

In [13]:
dict_confusion_matrix

{'a': 36, 'b': 12, 'c': 8, 'd': 1405}

In [14]:
get_all_scores(dict_confusion_matrix)

{'csi': 0.6428571428571429,
 'f': 0.008468595624558928,
 'far': 0.25,
 'gss': 0.6333935561577838,
 'h': 0.8181818181818182,
 'hss': 0.7755553507235681,
 'pc': 0.9863107460643394,
 'pss': 0.8097132225572593}

### Plotting results 

In [15]:
# getting complete dataframe
atypical_full_df, prediction_full_df = get_complete_df(prediction_results, aed_results)

In [16]:
# applying verification to df
atypical_full_df = apply_ael_to_df(atypical_full_df, ael_results)

In [19]:
# plotting
path_plot = '/home/antorosi/Documents/Prediction/out/figures'
name_plot = 'full_period_mode2'

prediction_conso_plot(data_conso_df, atypical_full_df, prediction_full_df, dict_colnames_conso, path_plot, name_plot)

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x1,y2 ]
[ (3,1) x1,y3 ]
[ (4,1) x1,y4 ]



/local/home/antorosi/PycharmProjects/KERAS-TS-VENV/lib/python3.5/site-packages/plotly/offline/offline.py:459: UserWarning:

Your filename `/home/antorosi/Documents/Prediction/out/figures/full_period_mode2` didn't end with .html. Adding .html to the end of your file.



'file:///home/antorosi/Documents/Prediction/out/figures/full_period_mode2.html'